In [ ]:
pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 22.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import joblib, json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.metrics import classification_report, accuracy_score
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/법률모델링/divorce_topic_labeled_corrected.csv", encoding="utf-8-sig")

In [ ]:
df2 = pd.read_csv("/content/drive/MyDrive/법률모델링/divorce_topic_labeled_testVersion.csv", encoding="utf-8-sig")

In [ ]:
intent_map = {
    "가능 여부 판단": 0,
    "절차·방법": 1,
    "증거·입증": 2,
    "금액·산정": 3,
    "기간·시효": 4,
    "제재·구제수단": 5,
    "법적 근거·조문": 6,
    "개념·범위·기준": 7,
    "기타" :  -1}
topic_map = {
    "이혼 시 금전 문제 (재산분할·양육비)":0,
    "위자료 및 손해배상 청구":1,
    "부정행위 및 제3자 개입 책임":2,
    "부정행위의 법률적 근거":3,
    "일반 상담·상황 문의 (비법률적 질문)": 4}

In [ ]:
df2.columns

Index(['announce_date', 'input', 'output', 'is_divorce', 'input_processed',
       'text_combined', 'combined_processed', 'topic_label',
       'topic_probability', 'topic_name'],
      dtype='object')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
import joblib


X_intent = df1["input"]  # 원본
y_intent = df1["intent_name"]

X_topic = df1["input_processed"]  # 전처리본
y_topic = df1["topic_name"]


# ==============================
# ==============================
# 2️⃣ 하이퍼파라미터 탐색 함수
# ==============================
def optimize_model_intent(save_path):
    X_intent_train, X_intent_test, y_intent_train, y_intent_test = train_test_split(
        X_intent, y_intent, test_size=0.2, random_state=42, stratify=y_intent)

    pipe = Pipeline([
        ("tfidf", TfidfVectorizer()),
        ("clf", LinearSVC(C =1.2, class_weight="balanced")),
    ])

    # 💡 탐색 범위 (너무 넓히면 오래 걸림)
    param_grid = {
      "tfidf__analyzer": ["char", "char_wb"],     # word는 효과 거의 없음 (의미 기반 모델 아님)
      "tfidf__ngram_range": [(1, 2), (2, 4)],     # 3~6은 오히려 과적합 or 불필요하게 큼
      "tfidf__max_features": [3000, 6000],        # 10000은 거의 효과 없음
      "clf__C": [0.8, 1.0, 1.2, 1.5]              # 2.0은 차이 거의 없음
  }

    print("\n🧠 [law_intent] GridSearchCV 탐색 중...")
    grid_intent  = GridSearchCV(
        pipe,
        param_grid,
        cv=3,
        scoring="f1_weighted",
        n_jobs=-1,
        verbose=2
    )
    grid_intent.fit(X_intent_train, y_intent_train)



    best_intent = grid_intent.best_estimator_
    preds = best_intent.predict(X_intent_test)

    print("\n✅ [law_intent] 최적 조합 발견!")
    print(grid_intent.best_params_)
    acc = accuracy_score(y_intent_test, preds)
    print(f"📈 [law_intent] 정확도: {acc:.4f}")
    print("=== 분류 리포트 ===")
    print(classification_report(y_intent_test, preds))

    joblib.dump(best_intent, "/content/drive/MyDrive/intent_model_hybrid.joblib")
    print("💾 law_intent 모델 저장 완료 ✅")

    return best_intent


def optimize_model_topic(save_path):
    X_topic_train, X_topic_test, y_topic_train, y_topic_test = train_test_split(
        X_topic, y_topic, test_size=0.2, random_state=42, stratify=y_topic)


    pipe = Pipeline([
        ("tfidf", TfidfVectorizer()),
        ("clf", LinearSVC(C =1.2, class_weight="balanced")),
    ])

    # 💡 탐색 범위 (너무 넓히면 오래 걸림)
    param_grid = {
        "tfidf__analyzer": ["char", "char_wb"],     # word는 효과 거의 없음 (의미 기반 모델 아님)
        "tfidf__ngram_range": [(1, 2), (2, 4)],     # 3~6은 오히려 과적합 or 불필요하게 큼
        "tfidf__max_features": [3000, 6000],        # 10000은 거의 효과 없음
        "clf__C": [0.8, 1.0, 1.2, 1.5]              # 2.0은 차이 거의 없음
    }


    print("\n🧠 [law_topic] GridSearchCV 탐색 중...")
    grid_topic  = GridSearchCV(
        pipe,
        param_grid,
        cv=3,
        scoring="f1_weighted",
        n_jobs=-1,
        verbose=2
    )
    grid_topic.fit(X_topic_train, y_topic_train)



    best_topic = grid_topic.best_estimator_
    preds = best_topic.predict(X_topic_test)

    print("\n✅ [law_topic] 최적 조합 발견!")
    print(grid_topic.best_params_)
    acc = accuracy_score(y_topic_test, preds)
    print(f"📈 [law_topic] 정확도: {acc:.4f}")
    print("=== 분류 리포트 ===")
    print(classification_report(y_topic_test, preds))

    joblib.dump(best_topic, "/content/drive/MyDrive/topic_best.joblib")
    print("💾 law_topic 모델 저장 완료 ✅")

    return best_topic


# ==============================
# 3️⃣ intent + topic 모델 최적화
# ==============================
intent_model = optimize_model_intent("/content/drive/MyDrive/intent_model_hybrid.joblib")
topic_model  = optimize_model_topic("/content/drive/MyDrive/topic_best.joblib")

# ==============================
# 4️⃣ 예시 예측
# ==============================
examples = [
     "위자료 받을 수 있을까요?",
    "이혼 절차는 어떻게 되나요?",
    "배우자가 외도했어요.",
    "증거를 꼭 제출해야 하나요?",
    "이혼 후에도 재산분할이 가능한가요?",
    "양육비를 안 주면 어떻게 되나요?",
    "이혼 후에도 양육비 받을 수 있나요?",
    "정서적 배신도 부정행위로 인정되나요?",
    "상담은 화상으로도 가능할까요?",
]

print("\n=== 🔍 최적 모델 예측 결과 ===")
for q in examples:
    intent_pred = intent_model.predict([q])[0]
    topic_pred = topic_model.predict([preprocess_text(q)])[0]

    print(f"🗨️ {q}")
    print(f"➡️ 의도(law_intent): {intent_pred}")
    print(f"➡️ 주제(law_topic): {topic_pred}\n")



🧠 [law_intent] GridSearchCV 탐색 중...
Fitting 3 folds for each of 32 candidates, totalling 96 fits

✅ [law_intent] 최적 조합 발견!
{'clf__C': 1.2, 'tfidf__analyzer': 'char', 'tfidf__max_features': 3000, 'tfidf__ngram_range': (1, 2)}
📈 [law_intent] 정확도: 0.8118
=== 분류 리포트 ===
              precision    recall  f1-score   support

    가능 여부 판단       0.70      0.82      0.76        34
    개념·범위·기준       0.69      0.58      0.63        19
       금액·산정       0.80      0.87      0.83        23
       기간·시효       0.73      0.73      0.73        22
          기타       0.00      0.00      0.00         1
    법적 근거·조문       0.83      0.75      0.79        40
       절차·방법       0.92      0.77      0.84        57
     제재·구제수단       0.76      0.91      0.83        32
       증거·입증       0.90      0.93      0.92        59

    accuracy                           0.81       287
   macro avg       0.70      0.71      0.70       287
weighted avg       0.82      0.81      0.81       287

💾 law_intent 모델 저장 완료 ✅

🧠 

In [ ]:
intent_model = joblib.load("/content/drive/MyDrive/intent_model_hybrid.joblib")
topic_model = joblib.load("/content/drive/MyDrive/topic_best.joblib")

df2["pred_intent"] = intent_model.predict(df2["input"])
df2["pred_topic"] = topic_model.predict(df2["input"].apply(preprocess_text))
print("\n=== 🔍 최적 모델 예측 결과 ===")

# 상위 몇 개 미리보기
for i, row in df2.head(10).iterrows():  # 10개만 확인
    print(f"🗨️ {row['input']}")
    print(f"➡️ 의도(law_intent): {row['pred_intent']}")
    print(f"➡️ 주제(law_topic): {row['pred_topic']}\n")


=== 🔍 최적 모델 예측 결과 ===
🗨️ 이혼 시 재산분할 청구가 어느 정도 인정될 수 있나요?
➡️ 의도(law_intent): 가능 여부 판단
➡️ 주제(law_topic): 이혼 시 금전 문제 (재산분할·양육비)

🗨️ 혼인관계에서 부정행위를 저지른 배우자가 이혼 후 약정한 금액을 지급하지 않는 경우, 법적 조치는 어떠한가요?
➡️ 의도(law_intent): 법적 근거·조문
➡️ 주제(law_topic): 부정행위 및 제3자 개입 책임

🗨️ 이혼합의서 작성 후 이혼의 효력이 상실되었을 때, 재산분할에 관한 합의의 효력은 어떻게 되나요?
➡️ 의도(law_intent): 절차·방법
➡️ 주제(law_topic): 이혼 시 금전 문제 (재산분할·양육비)

🗨️ 배우자 있는 자와 간통행위를 한 사람(상간자)은 법적으로 어떤 책임이 있나요?
➡️ 의도(law_intent): 제재·구제수단
➡️ 주제(law_topic): 부정행위 및 제3자 개입 책임

🗨️ 동호회 활동 중 알게 된 사람과 부정행위가 사실혼 관계 파탄에 미치는 영향은 어떻게 평가되나요?
➡️ 의도(law_intent): 절차·방법
➡️ 주제(law_topic): 부정행위 및 제3자 개입 책임

🗨️ 이혼 소송 분야에서 일방의 부정행위가 있는 있음에도 위자료 청구가 인정되지 않을 수 있는 경우는  무엇인가요?
➡️ 의도(law_intent): 증거·입증
➡️ 주제(law_topic): 위자료 및 손해배상 청구

🗨️ 이혼 소송에서 위자료 청구와 위자료 지급은 어떤 기준에 의해 결정되나요?
➡️ 의도(law_intent): 증거·입증
➡️ 주제(law_topic): 위자료 및 손해배상 청구

🗨️ 부모가 자녀의 양육비를 지급할 의무를 부인하기 위해 증명해야 하는 조건은 무엇인가요?
➡️ 의도(law_intent): 증거·입증
➡️ 주제(law_topic): 이혼 시 금전 문제 (재산분할·양육비)

🗨️ 양육비 산정 기준에 있어 법원이 고려해야 할 사항은 무엇인가요?
➡️ 의도(law_inte

In [ ]:
examples = [
    "남편이 생활비를 주지 않고 집을 나갔어요. 이혼 사유가 될까요?",
    "결혼 전 모은 돈도 재산분할 대상이 되나요?",
    "배우자가 제 명의로 빚을 냈어요. 제가 갚아야 하나요?",
    "이혼 소송 중에 다른 사람을 만나면 문제가 되나요?",
    "상간녀한테 위자료 청구하려면 어떤 증거가 필요해요?",
    "아이 아빠가 양육비를 계속 안 주면 어떻게 해야 하나요?",
    "합의이혼을 하기로 했는데 상대가 갑자기 안 나오면 어떻게 되나요?",
    "이혼 판결 후에 위자료를 추가로 청구할 수 있나요?",
    "가정폭력이 있었는데 증거가 없어요. 이혼 가능할까요?",
    "상간남이 결혼한 사실을 몰랐다고 하면 책임이 없나요?",
    "이혼 후에 아이 성을 바꾸려면 어떻게 해야 하나요?",
    "법원 조정 절차가 꼭 필요한가요?",
    "이혼 후에도 남편이 저를 스토킹하면 처벌할 수 있나요?",
    "남편이 제 명의로 된 재산을 몰래 팔았어요. 돌려받을 수 있을까요?",
    "아내가 외도했는데 증거가 카톡뿐이에요. 위자료 받을 수 있을까요?",
    "상대방이 위자료를 주기로 약속했는데 안 지켜요. 강제집행 가능할까요?",
    "별거 중인데 이혼소송을 먼저 제기할 수 있나요?",
    "결혼생활이 짧아도 재산분할 받을 수 있나요?",
    "배우자가 정신병 진단을 받았어요. 이혼할 수 있나요?",
    "법률상 혼인신고만 되어 있고 사실상 별거 중이에요. 어떻게 해야 하나요?"
]

print("\n=== 🔍 현장형 질의 예측 결과 ===")
for q in examples:
    intent_pred = intent_model.predict([q])[0]
    topic_pred = topic_model.predict([preprocess_text(q)])[0]
    print(f"🗨️ {q}")
    print(f"➡️ 의도(law_intent): {intent_pred}")
    print(f"➡️ 주제(law_topic): {topic_pred}\n")



=== 🔍 현장형 질의 예측 결과 ===
🗨️ 남편이 생활비를 주지 않고 집을 나갔어요. 이혼 사유가 될까요?
➡️ 의도(law_intent): 가능 여부 판단
➡️ 주제(law_topic): 부정행위의 법률적 근거

🗨️ 결혼 전 모은 돈도 재산분할 대상이 되나요?
➡️ 의도(law_intent): 가능 여부 판단
➡️ 주제(law_topic): 이혼 시 금전 문제 (재산분할·양육비)

🗨️ 배우자가 제 명의로 빚을 냈어요. 제가 갚아야 하나요?
➡️ 의도(law_intent): 법적 근거·조문
➡️ 주제(law_topic): 부정행위의 법률적 근거

🗨️ 이혼 소송 중에 다른 사람을 만나면 문제가 되나요?
➡️ 의도(law_intent): 법적 근거·조문
➡️ 주제(law_topic): 부정행위의 법률적 근거

🗨️ 상간녀한테 위자료 청구하려면 어떤 증거가 필요해요?
➡️ 의도(law_intent): 증거·입증
➡️ 주제(law_topic): 위자료 및 손해배상 청구

🗨️ 아이 아빠가 양육비를 계속 안 주면 어떻게 해야 하나요?
➡️ 의도(law_intent): 절차·방법
➡️ 주제(law_topic): 이혼 시 금전 문제 (재산분할·양육비)

🗨️ 합의이혼을 하기로 했는데 상대가 갑자기 안 나오면 어떻게 되나요?
➡️ 의도(law_intent): 절차·방법
➡️ 주제(law_topic): 부정행위의 법률적 근거

🗨️ 이혼 판결 후에 위자료를 추가로 청구할 수 있나요?
➡️ 의도(law_intent): 증거·입증
➡️ 주제(law_topic): 위자료 및 손해배상 청구

🗨️ 가정폭력이 있었는데 증거가 없어요. 이혼 가능할까요?
➡️ 의도(law_intent): 증거·입증
➡️ 주제(law_topic): 부정행위의 법률적 근거

🗨️ 상간남이 결혼한 사실을 몰랐다고 하면 책임이 없나요?
➡️ 의도(law_intent): 가능 여부 판단
➡️ 주제(law_topic): 부정행위의 법률적 근거

🗨️ 이혼 후에 아이 성을 바꾸려면 어떻게 해야 하나요?
➡

In [ ]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

# 형태소 분석기 초기화
okt = Okt()
# ✅ 1️⃣ 형태소 단위 tokenizer 함수
def tokenize_okt(text):
    return [token for token, pos in okt.pos(text, stem=True)
            if pos not in ['Josa', 'Eomi', 'Punctuation']]  # 조사, 어미 제거


# ✅ 3️⃣ TF-IDF 벡터화
vectorizer = TfidfVectorizer(
    tokenizer=tokenize_okt,       # 🔥 형태소 분석기 연결
    ngram_range=(1, 2),
    max_features=3000,
)

X = vectorizer.fit_transform(df["input"])
feature_names = vectorizer.get_feature_names_out()

# ✅ 4️⃣ 토픽별 키워드 뽑기
def get_top_keywords_per_topic(df, X, label_col, n_keywords=15):
    topics = {}
    for topic in df[label_col].unique():
        idx = df[df[label_col] == topic].index
        topic_tfidf = X[idx].mean(axis=0)
        top_indices = topic_tfidf.A1.argsort()[::-1][:n_keywords]
        topics[topic] = [feature_names[i] for i in top_indices]
    return topics

topic_keywords = get_top_keywords_per_topic(df, X, 'topic_name', 15)

# ✅ 5️⃣ 결과 출력
for topic, keywords in topic_keywords.items():
    print(f"\n=== 🏷️ {topic} ===")
    print(", ".join(keywords))



/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(



=== 🏷️ 위자료 및 손해배상 청구 ===
위자료, 청구, 손해배상, 부정행위, 하다, 손해배상 청구, 인하다, 무엇, 적, 정신, 정신 적, 부정행위 인하다, 위자료 청구, 되다, 배우자

=== 🏷️ 부정행위의 법률적 근거 ===
이혼, 되다, 어떻다, 있다, 하다, 하나요, 시효, 사유, 증거, 무엇, 법적, 소송, 이혼 사유, 인정, 법원

=== 🏷️ 부정행위 및 제3자 개입 책임 ===
제, 3, 제 3, 하다, 부정행위, 책임, 부부, 3 자가, 자가, 경우, 무엇, 되다, 부정행위 하다, 손해배상 책임, 조건

=== 🏷️ 이혼 시 금전 문제 (재산분할·양육비) ===
재산, 재산 분할, 분할, 양육비, 되다, 있다, 어떻다, 하다, 이혼, 무엇, 분할 청구, 청구, 시, 후, 따르다

=== 🏷️ 일반 상담·상황 문의 (비법률적 질문) ===
상담, 받다, 있다, 상담 받다, 수 있다, 수, 받다 수, 가능하다, 하다, 되다, 법률 상담, 상담 중, 상담 후, 후, 이혼


In [ ]:
# --------------------------
# 2️⃣ TF-IDF용 토크나이저 함수
# --------------------------
from konlpy.tag import Okt
import re

okt = Okt()

def tokenize_okt(text):
    """정제 + 형태소 분석 + 중복 최소화 버전"""
    if not isinstance(text, str):
        text = str(text)

    # 0️⃣ 제3자 변형 통합
    text = re.sub(r'제\s*\d+\s*자', lambda m: m.group(0).replace(" ", ""), text)

    # 1️⃣ 특수문자 제거 (숫자 유지)
    text = re.sub('[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', text)

    # 2️⃣ 형태소 분석 (어간 복원)
    tokens = okt.morphs(text, stem=True)

    # 3️⃣ “제3자” 합치기 보정
    merged_tokens = []
    i = 0
    while i < len(tokens):
        if (
            i + 2 < len(tokens)
            and tokens[i] == "제"
            and tokens[i + 1].isdigit()
            and tokens[i + 2] == "자"
        ):
            merged_tokens.append(f"제{tokens[i+1]}자")
            i += 3
        else:
            merged_tokens.append(tokens[i])
            i += 1

    # 4️⃣ 불용어 제거 + 법률키워드 보존
    filtered = []
    for t in merged_tokens:
        if t in LEGAL_KEYWORDS:  # 법률 키워드면 무조건 살림
            filtered.append(t)
        elif t not in MINIMAL_STOPWORDS and len(t) > 1:
            filtered.append(t)

    # 5️⃣ 중복 제거 (순서 유지)
    final_tokens = list(dict.fromkeys(filtered))

    return final_tokens




vectorizer = TfidfVectorizer(
    tokenizer=tokenize_okt,    # ← 우리가 만든 형태소 분석기
    token_pattern=None,        # ← 정규식 끄기 (안 끄면 또 쪼갬)
    ngram_range=(1, 2),
    max_features=3000
)

X = vectorizer.fit_transform(df1["input_processed"])
feature_names = vectorizer.get_feature_names_out()
# ==============================
# 3️⃣ 토픽별 키워드 추출 함수
# ==============================
def get_top_keywords_per_topic(df, X, label_col, n_keywords=15):
    topics = {}
    for topic in df[label_col].unique():
        idx = df[df[label_col] == topic].index
        topic_tfidf = X[idx].mean(axis=0)
        top_indices = topic_tfidf.A1.argsort()[::-1][:n_keywords]
        topics[topic] = [feature_names[i] for i in top_indices]
    return topics

topic_keywords = get_top_keywords_per_topic(df1, X, 'topic_name', 15)

In [ ]:
if 'topic_keywords' in df1.columns:
    df1['topic_keywords'] = df1['topic_keywords'].apply(
        lambda x: ', '.join(x) if isinstance(x, (list, tuple)) else str(x)
    )

In [ ]:
topic_keywords_map = {
    '부정행위의 법률적 근거': [
        '이혼', '법적', '사유', '시효', '혼인', '이혼 사유', '소송',
        '기준', '증거', '법원', '중단', '파탄', '효력', '부정행위', '판단'
    ],

    '위자료 및 손해배상 청구': [
        '위자료', '청구', '손해배상', '부정행위', '손해배상 청구', '정신',
        '위자료 청구', '배우자', '부정행위 정신', '배우자 부정행위',
        '정신 고통', '고통', '부정행위 손해배상', '기준', '결정'
    ],

    '부정행위 및 제3자 개입 책임': [
        '제3자', '부정행위', '책임', '부부', '조건', '손해배상 책임',
        '불법행위', '배우자', '손해배상', '조건 제3자', '일방',
        '부부 일방', '법적', '일방 부정행위', '생활'
    ],

    '이혼 시 금전 문제 (재산분할·양육비)': [
        '재산', '분할', '재산 분할', '양육비', '이혼', '분할 청구',
        '청구', '이혼 재산', '기준', '자녀', '사실혼', '법적',
        '고려', '계산', '비율'
    ],

    '일반 상담·상황 문의 (비법률적 질문)': [
        '상담', '받다', '상담 받다', '법률 상담', '법률', '이혼',
        '이혼 상담', '진행', '바로', '감정', '예약', '다른',
        '변호사', '서류', '상담 신청'
    ]
}


df1['topic_keywords'] = df1['topic_name'].map(topic_keywords_map)

In [ ]:
# 리스트를 문자열로 변환해서 저장
df1['topic_keywords'] = df1['topic_keywords'].apply(
    lambda x: ', '.join(x) if isinstance(x, (list, tuple)) else str(x)
)


In [ ]:
df1.to_csv("/content/drive/MyDrive/divorce_topic_labeled_with_topic_keywords.csv", encoding="utf-8-sig", index=False)
print("✅ CSV 저장 완료! → law_qa_with_topic_keywords.csv")

✅ CSV 저장 완료! → law_qa_with_topic_keywords.csv


In [ ]:
# 콘솔 미리보기
for topic, keywords in topic_keywords.items():
    print(f"\n=== 🏷️ {topic} ===")
    print(", ".join(keywords))


=== 🏷️ 부정행위의 법률적 근거 ===
이혼, 법적, 사유, 시효, 혼인, 이혼 사유, 소송, 기준, 증거, 법원, 중단, 파탄, 효력, 부정행위, 판단

=== 🏷️ 위자료 및 손해배상 청구 ===
위자료, 청구, 손해배상, 부정행위, 손해배상 청구, 정신, 위자료 청구, 배우자, 부정행위 정신, 배우자 부정행위, 정신 고통, 고통, 부정행위 손해배상, 기준, 결정

=== 🏷️ 부정행위 및 제3자 개입 책임 ===
제3자, 부정행위, 책임, 부부, 조건, 손해배상 책임, 불법행위, 배우자, 손해배상, 조건 제3자, 일방, 부부 일방, 법적, 일방 부정행위, 생활

=== 🏷️ 이혼 시 금전 문제 (재산분할·양육비) ===
재산, 분할, 재산 분할, 양육비, 이혼, 분할 청구, 청구, 이혼 재산, 기준, 자녀, 사실혼, 법적, 고려, 계산, 비율

=== 🏷️ 일반 상담·상황 문의 (비법률적 질문) ===
상담, 받다, 상담 받다, 법률 상담, 법률, 이혼, 이혼 상담, 진행, 바로, 감정, 예약, 다른, 변호사, 서류, 상담 신청


In [ ]:
# 위처럼 계속 전처리를 진행하지 않고 topic을 뽑아내니까 . 조사, 등등이 필요없는 것들이 나옴
# 단, 지금처럼 “LDA 모델링용”이 아니라 “TF-IDF 분류기 입력 전 전처리용”으로 씀
# 이렇게 적용하면 topic 혼동 문제(특히 ‘이혼’, ‘되다’ 같은 공통어) 해결됨.

In [ ]:
import re
from konlpy.tag import Okt

# MINIMAL_STOPWORDS와 LEGAL_KEYWORDS는 제공된 그대로 사용합니다.
MINIMAL_STOPWORDS = list(set([
    '것', '수', '때', '등', '들', '더', '이', '그', '저', '나', '우리', '같', '또', '만', '년', '월', '일', '하다', '있다', '되다',
    '가능하다', '가능', '가다', '되다', '하다', '있다', '없다', '않다', '된다', '한다', '어떻게', '어떤', '무엇', '언제', '어디서', '왜', '누가', '얼마', '몇',
    '알고', '싶다', '궁금하다', '문의', '질문', '답변', '설명', '이해', '과정', '절차', '이후', '다음', '먼저', '그리고', '그러나', '하지만', '그래서', '제자','제호','제조',
    '없', '있', '하', '되', '않', '나', '우리', '너', '당신', '같', '또', '것', '때', '등', '때문', '정도', '사실', '생각', '경우', '문제', '방법', '상황', '내용', '결과', '사람',
    '해야', '하면', '경우', '때는', '어느', '무슨', '어디', '누구' , ' 가지다' , '가지','하나요', '위해', '이혼',
    '대한', '관련', '따르다', '인정', '성립', '발생', '주장', '되나요', '있나요', '인가요', '할까요', '되나', '있나'
]))

LEGAL_KEYWORDS = [
    '위자료', '재산분할', '양육권', '친권', '면접교섭', '협의이혼', '청구', '배상', '손해', '책임',
    '차용금', '반환', '취소', '원상회복', '사해행위', '채권자', '배우자', '이혼사유', '이혼', '사유', '증명', '근거', '조건', '요건','기준','요소','범위','의무','효력','적용','판단',
    '혼인', '금전거래', '청구권', '액수', '정해지', '부적법', '혼인파탄', '파탄', '분할', '양육비', '면접',
    '교섭', '협의', '조정신청', '손해배상', '부부', '배우자', '당사자', '사람', '개인', '상대방',"입증"
]


def preprocess_text(text):
    protected_matches = {}

    def protect_term(match):
        # '제3자가' 같은 단어를 찾으면, 일단 그대로 두고 기록만 해둠
        original_word = match.group(0)
        # 고유 토큰을 만들 필요 없이, 원본 단어 자체를 기록
        if original_word not in protected_matches:
            protected_matches[original_word] = True
        return original_word

    def strip_josa(text):
        # 보호해야 할 특정 패턴 (예: '자의 의무')이 있으면 조사를 제거하지 않음
        preserve_patterns = [
            r'자의\s*의무', r'의\s*법적', r'의\s*효력', r'의\s*책임', r'의\s*근거',
            r'의\s*성격', r'의\s*의미', r'의\s*개념'
        ]
        for pattern in preserve_patterns:
            if re.search(pattern, text):
                return text

        # '제3자', '배우자' 등 핵심 단어 뒤에 붙은 조사를 먼저 제거
        # '를' 조사 추가
        text = re.sub(r'(제\d+자|배우자|친권자|청구자|상대방|부부|혼인|정조의무|부양의무)(의|가|는|은|을|를|와|과)$', r'\1', text)

        # 일반적인 명사 뒤 조사 제거
        return re.sub(r'([0-9가-힣]{2,})(의|가|를|은|는|과|와)$', r'\1', text)

    # 1단계: '제숫자+단어' 형태(예: 제3자, 제3자가)를 미리 찾아 기록
    text = re.sub(r'(제\d+[가-힣]+)', protect_term, text)

    # 2단계: 특수문자 제거
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', text)

    # 3단계: 형태소 분석
    okt = Okt()
    word_tokens = okt.pos(text, stem=True)

    meaningful_words = []

    # 4단계: 단어 필터링
    for word, pos in word_tokens:
        if word in LEGAL_KEYWORDS:
            meaningful_words.append(word)
        elif pos in ['Noun', 'Verb'] and len(word) > 1:
            token = strip_josa(word) # 형태소 분석된 단어에서 조사 제거
            if token not in MINIMAL_STOPWORDS:
                meaningful_words.append(token)

    # 5단계: 보호했던 단어 처리 (🚨핵심 수정 부분🚨)
    # 이 단계에서는 '제3자가'와 같은 단어가 목록에 있었다면,
    # 조사 제거 함수를 거쳐서 '제3자'로 만들어 추가합니다.
    for original_term in protected_matches.keys():
        processed_term = strip_josa(original_term)
        if processed_term not in meaningful_words: # 중복 방지
             meaningful_words.append(processed_term)

    # 6단계: 최종 결과 생성 (순서 유지를 위해 dict.fromkeys 사용)
    final_words = list(dict.fromkeys(meaningful_words))

    return ' '.join(final_words)
df_clean2['input_processed'] = df_clean2['input'].apply(preprocess_text)

In [ ]:
# 결과 비교를 위해 원본과 처리된 결과를 나란히 출력
print("\n=== 전처리 전/후 비교 ===")
for i in range(5):
    print(f"원본 [{i}]: {df_clean2['input'].iloc[i]}")
    print(f"결과 [{i}]: {df_clean2['input_processed'].iloc[i]}\n")


# 전처리된 데이터가 포함된 데이터프레임 확인
print("=== 각 목적에 맞게 생성된 전처리 컬럼들 ===")
print(df_clean2[['input', 'input_processed']].head())


=== 전처리 전/후 비교 ===
원본 [0]: 부정행위의 법적 정의는 무엇인가요?
결과 [0]: 부정행위 법적 정의

원본 [1]: 이혼 사유의 범위는 어디까지인가요?
결과 [1]: 이혼 사유 범위

원본 [2]: 위자료의 본질은 처벌인가요, 보상인가요?
결과 [2]: 위자료 본질 처벌 보상

원본 [3]: 정신적 손해배상의 법적 기준은 무엇인가요?
결과 [3]: 정신 손해배상 법적 기준

원본 [4]: 혼인의 본질적 의무에는 어떤 것이 포함되나요?
결과 [4]: 혼인 본질 의무 포함

=== 각 목적에 맞게 생성된 전처리 컬럼들 ===
                       input input_processed
0        부정행위의 법적 정의는 무엇인가요?      부정행위 법적 정의
1        이혼 사유의 범위는 어디까지인가요?        이혼 사유 범위
2     위자료의 본질은 처벌인가요, 보상인가요?    위자료 본질 처벌 보상
3    정신적 손해배상의 법적 기준은 무엇인가요?   정신 손해배상 법적 기준
4  혼인의 본질적 의무에는 어떤 것이 포함되나요?     혼인 본질 의무 포함


In [ ]:
df_clean2.loc[103]

,103
input,불법행위로 인한 손해배상 책임을 피하기 위해 제3자가 증명해야 하는 조건은 무엇인가요?
intent_name,증거·입증
intent_label,NaN
topic_name,부정행위 및 제3자 개입 책임
law_topic,2
output,"제3자가 불법행위로 인한 손해배상 책임을 피하기 위해서는 부정행위 당시 상대방이 정상적으로 혼인관계를 유지하는 배우자가 있음을 몰랐거나 알 수 없었다는 점을 증명해야 합니다. 예를 들어, 상대방으로부터 배우자가 있음에도 불구하고 그런 사실을 은폐했거나 속였다는 것을 증명할 수 있어야 합니다."
is_divorce,1.0
topic_probability,0.883175
announce_date,2021-08-11T09:00:00.000+09:00
law_intent,2
